In [1]:
import os
import re
import pandas as pd
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

def split_and_filter_text(text, min_length):
    # Разделение текста по строкам
    paragraphs = [para.strip() for para in text.split('\n') if para.strip()]
    
    filtered_paragraphs = []
    temp_paragraph = ""
    previous_paragraph = ""
    
    i = 0
    while i < len(paragraphs):
        para = paragraphs[i]
        # Проверка на наличие нумерации
        if re.match(r'^\d+(\.\s+|–\s+)', para):
            if previous_paragraph:
                # Объединяем предыдущий абзац с текущими нумерованными абзацами
                temp_paragraph = f"{previous_paragraph}\n{para}"
                previous_paragraph = ""  # Сброс предыдущего абзаца
            else:
                temp_paragraph = para
            # Продолжаем добавлять все следующие нумерованные абзацы
            while i + 1 < len(paragraphs) and re.match(r'^\d+(\.\s+|–\s+)', paragraphs[i + 1]):
                temp_paragraph += f"\n{paragraphs[i + 1]}"
                i += 1
            filtered_paragraphs.append(temp_paragraph)
            temp_paragraph = ""
        elif len(para) >= min_length:
            # Если уже есть сохранённый предыдущий абзац, добавляем его в filtered_paragraphs
            if previous_paragraph:
                filtered_paragraphs.append(previous_paragraph)
            previous_paragraph = para
        i += 1
    
    # Добавляем последний сохранённый абзац, если он не пустой
    if previous_paragraph:
        filtered_paragraphs.append(previous_paragraph)
    
    # Объединяем все отфильтрованные абзацы в один текст
    combined_text = "\n".join(filtered_paragraphs)
    
    return combined_text

def process_files_to_dataframe(input_dir, min_length):
    data = []
    
    for filename in os.listdir(input_dir):
        if filename.endswith('.docx'):
            input_file_path = os.path.join(input_dir, filename)
            
            # Чтение текста из файла
            text = read_docx(input_file_path)
            
            # Обработка текста
            combined_text = split_and_filter_text(text, min_length)
            
            # Добавление данных в список
            data.append({
                'name_doc': filename,
                'text': combined_text
            })
    
    # Создание DataFrame
    df = pd.DataFrame(data, columns=['name_doc', 'text'])
    return df

# Пример использования
input_directory = r'C:\Users\kamar\Desktop\word_files'
min_length = 60

pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

df = process_files_to_dataframe(input_directory, min_length)

df

,name_doc,text
0,document_1.docx,"В ходе выполнения регулярных технических работ по обновлению серверного оборудования 5 сентября 2023 года был выявлен ряд неполадок в системе резервного копирования данных. А именно, обнаружена сбойная работа программного обеспечения, ответственного за автоматическое создание резервных копий критически важных данных компании.\nСогласно графику, копирование должно осуществляться ежедневно в 02:00 ночи, однако с 1 по 5 сентября копии не создавались из-за ошибки в скриптах автоматизации. Ситуация была выявлена в процессе анализа логов системы после того, как несколько сотрудников обратились с жалобами на проблемы с доступом к старым версиям файлов.\n1. Обновлено программное обеспечение для резервного копирования;\n2. Выполнена полная проверка целостности существующих данных;\n3. Вручную созданы резервные копии всех критических данных.\nРекомендую провести аудит всей системы автоматизации резервного копирования, а также установить дополнительное программное обеспечение для мониторинга корректности выполнения процессов. Подробный отчет о выполненных действиях прилагается к докладной записке."
1,document_2.docx,"В связи с расширением отдела маркетинга и необходимостью введения новой должности для обеспечения роста клиентской базы и увеличения объёмов продаж, приказываю:\n1. Внести изменения в штатное расписание ООО «РеалИнвест», добавив новую должность ""Менеджер по работе с ключевыми клиентами"" с окладом в размере 120 000 рублей в месяц.\n2. Установить, что должность будет относиться к категории специалистов отдела маркетинга и будет подчиняться непосредственно начальнику отдела.\n3. Руководителю отдела кадров Мироновой В. С. внести необходимые изменения в штатное расписание и уведомить бухгалтерию о введении новой должности.\n4. Начальнику отдела маркетинга Лебедеву И. Н. подготовить описание должностных обязанностей для новой позиции и провести отбор кандидатов на вакантную должность до 30 сентября 2023 года.\n5. Контроль за выполнением настоящего приказа оставляю за собой."
2,document_3.docx,"В ходе проведения сверки расчетов с контрагентами по состоянию на 30 августа 2023 года была выявлена задолженность в размере 750 000 рублей от компании «ТехПром», по договору № 123/22 от 15 июня 2023 года. Согласно условиям договора, оплата должна была быть произведена не позднее 1 августа 2023 года, однако на сегодняшний день денежные средства не поступили.\nПосле направления нескольких уведомлений о просрочке платежа со стороны бухгалтерии ООО «ТехПром», ответа или подтверждения оплаты не последовало. В связи с этим, предлагаю направить официальное претензионное письмо в адрес контрагента с требованием о погашении задолженности в течение 10 рабочих дней с момента получения письма.\nПрошу вашего одобрения на данный шаг и, при необходимости, дальнейшие консультации с юридическим отделом для подготовки документов в суд в случае продолжения просрочки."


In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors

ds = Dataset.from_pandas(df)

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")
model = AutoModel.from_pretrained("ai-forever/ruRoberta-large")

device = torch.device("cpu")
model.to(device)

def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

embeddings_dataset = ds.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

C:\Users\kamar\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
import faiss

embeddings_dataset.add_faiss_index(column="embeddings")

question = "Из-за чего было обновлено программное обеспечение?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

  0%|          | 0/1 [00:00<?, ?it/s]

(1, 1024)

In [9]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=3,
)

In [10]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [11]:
for _, row in samples_df.iterrows():
    print(f"name_doc: {row.name_doc}")
    print(f"SCORE: {row.scores}")
    print(f"TEXT: {row.text}")    
    print("=" * 50)
    print()

name_doc: document_3.docx
SCORE: 1728.6524658203125
TEXT: В ходе проведения сверки расчетов с контрагентами по состоянию на 30 августа 2023 года была выявлена задолженность в размере 750 000 рублей от компании «ТехПром», по договору № 123/22 от 15 июня 2023 года. Согласно условиям договора, оплата должна была быть произведена не позднее 1 августа 2023 года, однако на сегодняшний день денежные средства не поступили.
После направления нескольких уведомлений о просрочке платежа со стороны бухгалтерии ООО «ТехПром», ответа или подтверждения оплаты не последовало. В связи с этим, предлагаю направить официальное претензионное письмо в адрес контрагента с требованием о погашении задолженности в течение 10 рабочих дней с момента получения письма.
Прошу вашего одобрения на данный шаг и, при необходимости, дальнейшие консультации с юридическим отделом для подготовки документов в суд в случае продолжения просрочки.

name_doc: document_1.docx
SCORE: 1711.770263671875
TEXT: В ходе выполнения регуля

In [12]:
samples_df

,name_doc,text,embeddings,scores
2,document_3.docx,"В ходе проведения сверки расчетов с контрагентами по состоянию на 30 августа 2023 года была выявлена задолженность в размере 750 000 рублей от компании «ТехПром», по договору № 123/22 от 15 июня 2023 года. Согласно условиям договора, оплата должна была быть произведена не позднее 1 августа 2023 года, однако на сегодняшний день денежные средства не поступили.\nПосле направления нескольких уведомлений о просрочке платежа со стороны бухгалтерии ООО «ТехПром», ответа или подтверждения оплаты не последовало. В связи с этим, предлагаю направить официальное претензионное письмо в адрес контрагента с требованием о погашении задолженности в течение 10 рабочих дней с момента получения письма.\nПрошу вашего одобрения на данный шаг и, при необходимости, дальнейшие консультации с юридическим отделом для подготовки документов в суд в случае продолжения просрочки.","[0.4015445113182068, 2.0305418968200684, -0.01710684597492218, -0.13322851061820984, 1.1900779008865356, 0.267352819442749, -0.14684385061264038, -0.3031570315361023, -0.754260241985321, -0.3564295768737793, 0.29723936319351196, 0.03971455991268158, -0.42879995703697205, -0.5431329011917114, -0.4921887516975403, 0.30604514479637146, 0.010111631825566292, -0.22432464361190796, 0.23077350854873657, -0.45458900928497314, 0.6047943830490112, -0.412834495306015, 0.19293178617954254, 0.07008355855941772, 0.04197201877832413, 0.469737708568573, -0.005845028907060623, -0.06388413906097412, -0.4199397563934326, -0.31704267859458923, 0.536045253276825, 0.7249675989151001, -0.17694661021232605, 0.12610715627670288, -0.7186512351036072, -0.02524355612695217, -0.436826229095459, -0.3663713037967682, -0.31223228573799133, -0.4265120029449463, 0.2101510912179947, 0.16188481450080872, -0.07586969435214996, 0.25179386138916016, 0.18370088934898376, -0.48141032457351685, 0.6803571581840515, -0.32218047976493835, -0.4304542541503906, -0.10080300271511078, -0.5718808174133301, 0.4174986183643341, -0.2631145119667053, 0.6147669553756714, -0.413236528635025, 0.4498494863510132, 0.13438892364501953, -0.3458822965621948, -0.04578873887658119, -0.021069809794425964, -0.542242705821991, 0.5669745802879333, -0.467386394739151, 0.15131831169128418, -0.3810306191444397, -0.3149951994419098, 0.6594238877296448, -0.13143667578697205, 0.7144100666046143, 0.6747872233390808, -0.2753923237323761, -0.013978064991533756, 0.06908492743968964, -0.7826055288314819, 0.4512442946434021, 0.3271233141422272, -0.11091203987598419, 0.6033533215522766, -0.7018834352493286, 0.0636560320854187, -0.37260136008262634, 0.014665152877569199, -0.4464547038078308, -0.5487155914306641, 0.5344093441963196, 0.35453444719314575, 0.9660980701446533, 1.3875727653503418, 0.314107209444046, 0.19106894731521606, 0.3045041263103485, -0.0591152161359787, -0.052829939872026443, 0.22737625241279602, -0.14511141180992126, -0.07860083878040314, -0.31293827295303345, 0.21767979860305786, -0.4356929659843445, -0.12020552903413773, ...]",1728.652466
1,document_1.docx,"В ходе выполнения регулярных технических работ по обновлению серверного оборудования 5 сентября 2023 года был выявлен ряд неполадок в системе резервного копирования данных. А именно, обнаружена сбойная работа программного обеспечения, ответственного за автоматическое создание резервных копий критически важных данных компании.\nСогласно графику, копирование должно осуществляться ежедневно в 02:00 ночи, однако с 1 по 5 сентября копии не создавались из-за ошибки в скриптах автоматизации. Ситуация была выявлена в процессе анализа логов системы после того, как несколько сотрудников обратились с жалобами на проблемы с доступом к старым версиям файлов.\n1. Обновлено программное обеспечение для резервного копирования;\n2. Выполнена полная проверка целостности существующих данных;\n3. Вручную созданы резервные копии всех критических данных.\nРекомендую провести аудит всей системы автоматизации резервного копирования, а также установить дополнит

In [16]:
!pip install sentence_transformers -q

In [17]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [18]:
# Загружаем модель для создания эмбеддингов предложений
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\kamar\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kamar\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

C:\Users\kamar\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
model.save(r'C:\Users\kamar\Desktop\NLP_VTB\paraphrase-multilingual-MiniLM-L12-v2')

In [19]:
# Ваши тексты датасета
texts = [
    '''В ходе выполнения регулярных технических работ по обновлению серверного оборудования 5 сентября 2023 года был выявлен ряд неполадок в системе резервного копирования данных. А именно, обнаружена сбойная работа программного обеспечения, ответственного за автоматическое создание резервных копий критически важных данных компании.\nСогласно графику, копирование должно осуществляться ежедневно в 02:00 ночи, однако с 1 по 5 сентября копии не создавались из-за ошибки в скриптах автоматизации. Ситуация была выявлена в процессе анализа логов системы после того, как несколько сотрудников обратились с жалобами на проблемы с доступом к старым версиям файлов.\n1. Обновлено программное обеспечение для резервного копирования;\n2. Выполнена полная проверка целостности существующих данных;\n3. Вручную созданы резервные копии всех критических данных.\nРекомендую провести аудит всей системы автоматизации резервного копирования, а также установить дополнительное программное обеспечение для мониторинга корректности выполнения процессов. Подробный отчет о выполненных действиях прилагается к докладной записке.''',
    '''В связи с расширением отдела маркетинга и необходимостью введения новой должности для обеспечения роста клиентской базы и увеличения объёмов продаж, приказываю:\n1. Внести изменения в штатное расписание ООО «РеалИнвест», добавив новую должность "Менеджер по работе с ключевыми клиентами" с окладом в размере 120 000 рублей в месяц.\n2. Установить, что должность будет относиться к категории специалистов отдела маркетинга и будет подчиняться непосредственно начальнику отдела.\n3. Руководителю отдела кадров Мироновой В. С. внести необходимые изменения в штатное расписание и уведомить бухгалтерию о введении новой должности.\n4. Начальнику отдела маркетинга Лебедеву И. Н. подготовить описание должностных обязанностей для новой позиции и провести отбор кандидатов на вакантную должность до 30 сентября 2023 года.\n5. Контроль за выполнением настоящего приказа оставляю за собой.''',
    '''В ходе проведения сверки расчетов с контрагентами по состоянию на 30 августа 2023 года была выявлена задолженность в размере 750 000 рублей от компании «ТехПром», по договору № 123/22 от 15 июня 2023 года. Согласно условиям договора, оплата должна была быть произведена не позднее 1 августа 2023 года, однако на сегодняшний день денежные средства не поступили.\nПосле направления нескольких уведомлений о просрочке платежа со стороны бухгалтерии ООО «ТехПром», ответа или подтверждения оплаты не последовало. В связи с этим, предлагаю направить официальное претензионное письмо в адрес контрагента с требованием о погашении задолженности в течение 10 рабочих дней с момента получения письма.\nПрошу вашего одобрения на данный шаг и, при необходимости, дальнейшие консультации с юридическим отделом для подготовки документов в суд в случае продолжения просрочки.''',
    # Добавьте больше текстов
]

# Создаем эмбеддинги для всех текстов
embeddings = model.encode(texts, convert_to_numpy=True)

# Создаем индекс FAISS
dimension = embeddings.shape[1]  # Размерность эмбеддингов
index = faiss.IndexFlatL2(dimension)  # Используем L2 (евклидову метрику)
index.add(embeddings)  # Добавляем эмбеддинги в индекс

C:\Users\kamar\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [20]:
# Ваш запрос
query = "технические работы по обновлению"

# Преобразуем запрос в вектор
query_embedding = model.encode([query], convert_to_numpy=True)

# Ищем ближайшие тексты (k = количество результатов)
k = 3  # Найти топ-3 ближайших текста
distances, indices = index.search(query_embedding, k)

# Выводим результаты
print(f"Запрос: {query}")
for i, idx in enumerate(indices[0]):
    print(f"Результат {i+1}: {texts[idx]} (дистанция: {distances[0][i]})")


Запрос: технические работы по обновлению
Результат 1: В ходе выполнения регулярных технических работ по обновлению серверного оборудования 5 сентября 2023 года был выявлен ряд неполадок в системе резервного копирования данных. А именно, обнаружена сбойная работа программного обеспечения, ответственного за автоматическое создание резервных копий критически важных данных компании.
Согласно графику, копирование должно осуществляться ежедневно в 02:00 ночи, однако с 1 по 5 сентября копии не создавались из-за ошибки в скриптах автоматизации. Ситуация была выявлена в процессе анализа логов системы после того, как несколько сотрудников обратились с жалобами на проблемы с доступом к старым версиям файлов.
1. Обновлено программное обеспечение для резервного копирования;
2. Выполнена полная проверка целостности существующих данных;
3. Вручную созданы резервные копии всех критических данных.
Рекомендую провести аудит всей системы автоматизации резервного копирования, а также установить дополнител